# 1. Имплементация модели

In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
from IPython.display import display

In [2]:
class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        sp = self.speaker().T
        return rownorm(sp * self.prior)

def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T

def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)

def display_reference_game(mod):
    d = mod.lexicon.copy()
    d['costs'] = mod.costs
    d.loc['prior'] = list(mod.prior) + [""]
    d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
    display(d)

# 2. Референциальная игра

## 2.1. Исходная игра

In [3]:
lex = pd.DataFrame([[1, 1, 0],
                   [0, 0, 1],
                   [0, 1, 0]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0], alpha=1)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,1.0,0.0,0
очки,0.000000,0.0,1.0,0
усы,0.000000,1.0,0.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.75,0.25,0.0
очки,0.00,0.00,1.0
усы,0.00,1.00,0.0


Да, из этого распределения выводится скалярная импликатура, так как существует иерархия "усы > шляпа", то есть, "усы" информативнее "шляпы", так как тем ситауциям, которым соответствует "усы", соответствует и "шляпа" (r2), но не наоборот (так, например ситуации r1 соответствует тольуо шляпа). Иначе говоря, сообщение "усы" описывают только ситуацию r2, а "шляпа" -- и r1, и r2, поэтому "усы" -- более сильное высказывание. Таким образом появляется скалярная импликатура "шляпа -> не усы", которая отражена в таблице прагматического слушающего преобладанием вероятности ситуации r1 в случае получения сообщения "шляпа".

## 2.2. Мена вероятностей

In [4]:
lex = pd.DataFrame([[1, 1, 0],
                   [0, 0, 1],
                   [0, 1, 0]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0, 0, 0], alpha=1)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.0,1.0,0.0,0
очки,0.0,0.0,1.0,0
усы,0.0,1.0,0.0,0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.330579,0.669421,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0


В данном случае импликатура не вычисляется из-за того, что ситуация r1 в целом маловероятна.

# 3. Игры с двумя импликатурами

## 3.1. Признаки на одной шкале

In [5]:
lex = pd.DataFrame([[1, 1, 1],
                   [0, 1, 1],
                   [0, 0, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0], alpha=1)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,1.0,1.0,0
очки,0.000000,1.0,1.0,0
усы,0.000000,0.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.632184,0.252874,0.114943
очки,0.000000,0.687500,0.312500
усы,0.000000,0.000000,1.000000


В данном случае у нас формируется одна иерархия, включающая в себя все признаки, в данном случае "усы > очки > шляпа". Поэтому "шляпа" по импликатуре приписывается только ситуации, где нет других аттрибутов, то есть, нельзя сказать о ситуации большего, "очки" - той ситуации, в которой также присутствует шляпа, но точно нет усов, а "усы" - единственной ситуации с усами.

### 3.1.1. Цена высказывания

In [6]:
lex = pd.DataFrame([[1, 1, 1],
                   [0, 1, 1],
                   [0, 0, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[5, 0, 0], alpha=1)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,1.0,1.0,5
очки,0.000000,1.0,1.0,0
усы,0.000000,0.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.337773,0.334394,0.327833
очки,0.000000,0.504953,0.495047
усы,0.000000,0.000000,1.000000


В таком случае прагматический слушающий не вычислит импликатуру, так как вряд ли говорящий вообще будет употреблять выражение "шляпа", если оно значительно дороже двух других.

### 3.1.2. Исходная вероятность

In [7]:
lex = pd.DataFrame([[1, 1, 1],
                   [0, 1, 1],
                   [0, 0, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[0.1, 0.1, 0.8], costs=[0, 0, 0], alpha=1)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.0,1.0,1.0,0
очки,0.0,1.0,1.0,0
усы,0.0,0.0,1.0,0
prior,0.1,0.1,0.8,
alpha,1.0,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.259481,0.122912,0.617607
очки,0.000000,0.165981,0.834019
усы,0.000000,0.000000,1.000000


Здесь также не происходит вычисления импликатуры, так как все ситуации, кроме r3, оказываются маловероятными.

### 3.1.3. Оптимальность

In [8]:
lex = pd.DataFrame([[1, 1, 1],
                   [0, 1, 1],
                   [0, 0, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0], alpha=5)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,1.0,1.0,0
очки,0.000000,1.0,1.0,0
усы,0.000000,0.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,5.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.892588,0.103865,0.003548
очки,0.000000,0.966971,0.033029
усы,0.000000,0.000000,1.000000


В случае выше мы видим более радикальную ситуацию, в которой явно прослеживается изначальная импликатура.

In [9]:
lex = pd.DataFrame([[1, 1, 1],
                   [0, 1, 1],
                   [0, 0, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0], alpha=1e-15)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000e+00,1.0,1.0,0
очки,0.000000e+00,1.0,1.0,0
усы,0.000000e+00,0.0,1.0,0
prior,3.333333e-01,0.333333,0.333333,
alpha,1.000000e-15,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.545455,0.272727,0.181818
очки,0.000000,0.600000,0.400000
усы,0.000000,0.000000,1.000000


Здесь, напротив, импликатура становится менее заметной, но при этом стоит отметить, что не исчезает совсем.

## 3.2. Признаки на разных шкалах

In [10]:
lex = pd.DataFrame([[1, 0, 0],
                   [1, 1, 0],
                   [0, 1, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0], alpha=1)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,0.0,0.0,0
очки,1.000000,1.0,0.0,0
усы,0.000000,1.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,1.0,0.000000,0.000000
очки,0.4,0.600000,0.000000
усы,0.0,0.333333,0.666667


Здесь есть две шкалы: "шляпа > очки" и "очки > усы", потому что "шляпа" информативнее "очков" (так как описывает только ситуацию r1), поэтому "очки" не описывают r1, ведь для него есть более сильное выражение. Тогда "очки" приписываются r2, и оказываются инфоррмативнее "усов", которые приписываются только r3.

### 3.2.1. Цена высказывания

In [11]:
lex = pd.DataFrame([[1, 0, 0],
                   [1, 1, 0],
                   [0, 1, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 5, 0], alpha=1)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,0.0,0.0,0
очки,1.000000,1.0,0.0,5
усы,0.000000,1.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,1.000000,0.000000,0.000000
очки,0.498332,0.501668,0.000000
усы,0.000000,0.006648,0.993352


В таком случае для "очков" не высчитывается импликатура, потому что не ожидается, что говорящий будет использовать это выражение, так как оно дороже других.

### 3.2.2. Исходная вероятность

In [12]:
lex = pd.DataFrame([[1, 0, 0],
                   [1, 1, 0],
                   [0, 1, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[0.1, 0.8, 0.1], costs=[0, 0, 0], alpha=1)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.0,0.0,0.0,0
очки,1.0,1.0,0.0,0
усы,0.0,1.0,1.0,0
prior,0.1,0.8,0.1,
alpha,1.0,,,



Pragmatic listener


,r1,r2,r3
шляпа,1.00000,0.00000,0.0
очки,0.02439,0.97561,0.0
усы,0.00000,0.80000,0.2


Здесь не высчитывается импликатура для "усов", потому что r3 оказывается намного менее вероятной, чем r2.

### 3.2.3. Оптимальность

In [13]:
lex = pd.DataFrame([[1, 0, 0],
                   [1, 1, 0],
                   [0, 1, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0], alpha=5)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,0.0,0.0,0
очки,1.000000,1.0,0.0,0
усы,0.000000,1.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,5.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,1.000000,0.000000,0.000000
очки,0.057143,0.942857,0.000000
усы,0.000000,0.333333,0.666667


Как и в прошлом случае, здесь более явно видно импликатуру.

In [14]:
lex = pd.DataFrame([[1, 0, 0],
                   [1, 1, 0],
                   [0, 1, 1]], index=['шляпа', 'очки', 'усы'],
                  columns=['r1', 'r2', 'r3'])
basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0], alpha=1e-15)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000e+00,0.0,0.0,0
очки,1.000000e+00,1.0,0.0,0
усы,0.000000e+00,1.0,1.0,0
prior,3.333333e-01,0.333333,0.333333,
alpha,1.000000e-15,,,



Pragmatic listener


,r1,r2,r3
шляпа,1.0,0.000000,0.000000
очки,0.5,0.500000,0.000000
усы,0.0,0.333333,0.666667


Здесь уже не высчитывается импликатура для "очков", правда все ещё заметна для "усов".

# 4. Нестандартные сферы для теории RSA

Мне показалось интересной мыслью попробовать применить теорию рационального речевого акта к дизамбигуации грамматических феноменов, например, относительных (таксисных) и абсолютных (дейксисных) времён в языках, где нет различия в морфологических формах, как в русском языке. <br>
Правда, довольно сложно найти контексты, где происходила бы реальная конкуренция, так как таксисные времена находятся обычно только в зависимых клаузах, как в "<i>Я думал, что он <b>ушёл</b>.</i>" ('<i>I thought he <b>had left</b>.</i>'). <br>
Но можно вызвать неоднозначность глаголами несовершенного вида: "<i>Я думал, что он <b>уходил</b>.</i>" ('<i>I thought he <b>was leaving</b> / <b>had been leaving</b>.</i>'). В данном случае prior будет практически целиком полагаться на контекст, а как более "дорогостоящие", но менее неоднозначные высказывания будут выступать высказывания с темпоральными адъюнктами: "<i>Я думал, что он <b>до этого уходил</b>.</i>" ('<i>I thought he <b>had been leaving before that</b></i>.') и "<i><b>Тогда</b> я думал, что он <b>уходил</b>.</i>" ('<i><b>Then</b> I thought he <b>was leaving</b>.</i>').